In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import requests
import pandas as pd
from src.tables import get_machines
from src.utils import df_na_vals

In [3]:
raw = get_machines()

In [4]:
cpu_name = raw.cpu_name.str.replace(r'^\d\dth Gen ','')
cpu_name = cpu_name.str.replace(r'^AMD','')
cpu_name = cpu_name.str.replace(r'Threadripper','TR')
cpu_name = cpu_name.str.replace(r'Processor$','')
cpu_name = cpu_name.str.replace(r'-Core','C')
cpu_name = cpu_name.str.replace(r' with Radeon Graphics','')
# cpu_name = raw.cpu_name.str.replace('Core™', '')
raw.cpu_name = cpu_name
raw['cores'] = (raw.cpu_cores / raw.num_gpus).fillna(0).astype(int)
raw['ram'] = (raw.cpu_ram / raw.num_gpus / 1024).round().astype(int)
raw['dph'] = raw.dph_base / raw.num_gpus
raw['disk'] = (raw.disk_space / raw.num_gpus).astype(int)
raw['rented_fraq'] = '[' + raw.num_gpus_rented.astype(str) + "/" + raw.num_gpus.astype(str) + ']'
raw['country'] = raw.location.dropna().apply(lambda loc: loc['country'])
# raw['rented'] = raw.num_gpus_rented > 0
raw['rented'] = raw.num_gpus_rented == raw.num_gpus
raw['min_fraq'] = raw.min_chunk/raw.num_gpus

raw.dlperf = raw.dlperf.round(1)
raw.dlperf_per_dphtotal = raw.dlperf_per_dphtotal.round(1)
raw.disk_bw = raw.disk_bw.round()
raw.duration = pd.to_timedelta(raw.duration * 1e9).dt.days
raw.inet_down = raw.inet_down.fillna(0).astype(int)
raw.inet_up_cost = (raw.inet_up_cost * 1000).fillna(0).astype(int)
raw.inet_down_cost = (raw.inet_down_cost * 1000).fillna(0).astype(int)


# raw.end_date = (pd.to_datetime(raw.end_date * 1e9) - pd.Timestamp.now()).dt.days
raw.end_date = pd.to_datetime(raw.end_date * 1e9)
raw.start_date = pd.to_datetime(raw.start_date * 1e9)
raw.set_index('machine_id', inplace=True)

C:\Users\ergot\AppData\Local\Temp\ipykernel_9688\1575474732.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  cpu_name = raw.cpu_name.str.replace(r'^\d\dth Gen ','')
C:\Users\ergot\AppData\Local\Temp\ipykernel_9688\1575474732.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  cpu_name = cpu_name.str.replace(r'^AMD','')
C:\Users\ergot\AppData\Local\Temp\ipykernel_9688\1575474732.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  cpu_name = cpu_name.str.replace(r'Processor$','')


In [5]:
cols = ['gpu_name',  'rented_fraq', 'dph', 'credit_discount_max', 'dlperf_per_dphtotal', 'cores', 'ram', 'disk', 'disk_bw',
         'min_chunk', 'country', 'storage_cost', 'reliability2', 'duration', 'inet_up', 'inet_down', 'inet_up_cost', 'inet_down_cost', 'cuda_max_good', 'driver_version',  'pci_gen', 'gpu_lanes', 'hosting_type',  'pcie_bw', 'cpu_name', 'mobo_name',  'location']

raw.query('dph >= 0.5')[cols].sort_values(by='dph', ascending=False)

,gpu_name,rented_fraq,dph,credit_discount_max,dlperf_per_dphtotal,cores,ram,disk,disk_bw,min_chunk,...,inet_down_cost,cuda_max_good,driver_version,pci_gen,gpu_lanes,hosting_type,pcie_bw,cpu_name,mobo_name,location
machine_id,,,,,,,,,,,,,,,,,,,,,
6767,RTX A6000,[0/2],5.01,0.4,5.0,10,126,1215,2833.0,1,...,10,12.0,525.125.06,3,16,NaN,11.5,Core™ i9-10900X,WS X299 SAGE,"{'country': 'CA', 'location': 'Vancouver, Brit..."
11287,RTX 4090,[0/1],5.00,0.2,12.1,40,189,1948,519.0,1,...,14,12.0,525.116.04,3,16,NaN,10.8,Xeon® E5-2680 v2,X9DRH,"{'country': 'US', 'location': 'Fort Worth, Tex..."
11937,H100 PCIE,[2/2],3.00,0.4,33.8,64,118,1374,4319.0,1,...,0,12.2,535.54.03,4,16,NaN,19.8,EPYC 7742 64C,MZ52,"{'country': 'US', 'location': 'Ogden, Utah', '..."
6062,A100 SXM4,[8/8],2.00,0.4,30.0,32,126,8,3733.0,4,...,0,12.2,535.54.03,4,16,NaN,10.3,EPYC 7763 64C,H12DGO,"{'country': 'US', 'lat': 37.751, 'long': -97.8..."
6063,A100 SXM4,[0/8],2.00,0.4,29.6,32,126,140,6251.0,4,...,0,12.0,525.85.12,4,16,NaN,14.7,EPYC 7763 64C,H12DGO,"{'country': 'US', 'lat': 37.751, 'long': -97.8..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11353,RTX A6000,[1/10],0.50,0.4,41.1,1,50,38,1294.0,1,...,2,12.2,535.54.03,3,16,1.0,11.9,EPYC 7252 8C,H11DSi,"{'country': 'SE', 'location': 'Umeå, Västerbot..."
10225,A40,[4/5],0.50,0.4,57.8,3,50,29,1873.0,1,...,0,12.0,525.125.06,3,8,1.0,11.9,EPYC 7252 8C,Default string,"{'country': 'SE', 'location': 'Umeå, Västerbot..."
8937,A40,[4/5],0.50,0.4,57.0,3,50,58,1357.0,1,...,10,12.0,525.125.06,3,16,1.0,11.9,EPYC 7252 8C,Default string,"{'country': 'SE', 'location': 'Umeå, Västerbot..."


In [6]:
raw[cols].query('country == "RU"')

,gpu_name,rented_fraq,dph,credit_discount_max,dlperf_per_dphtotal,cores,ram,disk,disk_bw,min_chunk,...,inet_down_cost,cuda_max_good,driver_version,pci_gen,gpu_lanes,hosting_type,pcie_bw,cpu_name,mobo_name,location
machine_id,,,,,,,,,,,,,,,,,,,,,
12133,RTX A4000,[0/1],0.100,0.40,148.0,12,16,394,1695.0,1,...,2,12.2,535.54.03,3,16,NaN,12.5,Core™ i7-8700K,B360M,"{'country': 'RU', 'location': 'Moscow', 'lat':..."
12123,RTX 3090,[0/1],0.120,0.40,273.0,12,31,834,603.0,1,...,0,12.2,535.54.03,4,16,NaN,24.9,Ryzen 5 3600 6C,B550M DS3H,"{'country': 'RU', 'location': 'Moscow', 'lat':..."
11900,RTX 3090,[3/3],0.219,0.05,135.5,10,21,120,3227.0,1,...,0,12.0,525.125.06,4,4,NaN,5.9,Ryzen 9 5950X 16C,ROG STRIX X570,"{'country': 'RU', 'location': 'Maslyaninskiy, ..."
11637,RTX 3080 Ti,[2/3],0.160,0.05,155.0,10,42,406,3905.0,1,...,0,12.0,525.125.06,4,4,NaN,6.0,Ryzen 9 5950X 16C,ROG STRIX X570,"{'country': 'RU', 'location': 'Maslyaninskiy, ..."
11613,RTX 3080 Ti,[2/2],0.160,0.05,156.2,18,63,535,3129.0,1,...,0,12.0,525.125.06,3,16,NaN,11.8,Xeon® E5-2696 v3,X99,"{'country': 'RU', 'location': 'Maslyaninskiy, ..."
11536,RTX 3090,[4/4],0.230,0.10,128.2,14,63,107,1105.0,2,...,0,12.0,525.105.17,3,16,NaN,10.1,Xeon® E5-2680 v4,Z10PG,"{'country': 'RU', 'location': 'Moscow Oblast',..."
11480,RTX 3090,[2/2],0.198,0.05,145.9,16,63,84,2684.0,1,...,0,12.0,525.125.06,4,4,NaN,6.2,Ryzen 9 5950X 16C,ROG STRIX X570,"{'country': 'RU', 'location': 'Maslyaninskiy, ..."
11395,RTX 3090,[4/4],0.240,0.10,123.7,14,63,257,2311.0,2,...,2,12.0,525.125.06,3,16,NaN,10.6,Xeon® E5-2680 v4,Z10PG,"{'country': 'RU', 'location': 'Moscow Oblast',..."
11385,RTX 4090,[6/6],0.395,0.40,147.3,10,42,191,3625.0,1,...,0,12.0,525.105.17,4,16,NaN,23.6,EPYC 7532 32C,ROMED8,"{'country': 'RU', 'location': 'Moscow', 'lat':..."
